In [2]:
import json
import numpy as np
import skimage
import tifffile
import os
import shutil

In [5]:
root_dir = "/disk3/eric/dataset/Weapon_System/segmentation_pipes"

train_img_dir  = os.path.join(root_dir, "train/images")
train_ann_dir  = os.path.join(root_dir, "train/_annotations.coco.json")
train_mask_dir = os.path.join(root_dir, "train/labels")

valid_img_dir  = os.path.join(root_dir, "valid/images")
valid_ann_dir  = os.path.join(root_dir, "valid/_annotations.coco.json")
valid_mask_dir = os.path.join(root_dir, "valid/labels")

In [6]:
def category_change(file_path):
    '''
    coco segmentation format from roboflow
    '''

    with open(file_path, 'r') as f:
        data = json.load(f)

    #--------------------------------
    images = data['images']
    annotations = data['annotations']
    categories = data['categories']
    #--------------------------------
    categories[0]['name'] = "background"
    categories[0]['supercategory'] = "background"
    display(file_path)
    display(categories)
    
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)


#--
category_change(train_ann_dir)
category_change(valid_ann_dir)

'/disk3/eric/dataset/Weapon_System/segmentation_pipes/train/_annotations.coco.json'

[{'id': 0, 'name': 'background', 'supercategory': 'background'},
 {'id': 1, 'name': 'M2A1Slammer', 'supercategory': 'M2A1Slammer'},
 {'id': 2, 'name': 'M5SandstormMLRS', 'supercategory': 'M2A1Slammer'},
 {'id': 3, 'name': 'T140Angara', 'supercategory': 'M2A1Slammer'},
 {'id': 4, 'name': 'ZamakMRL', 'supercategory': 'M2A1Slammer'}]

'/disk3/eric/dataset/Weapon_System/segmentation_pipes/valid/_annotations.coco.json'

[{'id': 0, 'name': 'background', 'supercategory': 'background'},
 {'id': 1, 'name': 'M2A1Slammer', 'supercategory': 'M2A1Slammer'},
 {'id': 2, 'name': 'M5SandstormMLRS', 'supercategory': 'M2A1Slammer'},
 {'id': 3, 'name': 'T140Angara', 'supercategory': 'M2A1Slammer'},
 {'id': 4, 'name': 'ZamakMRL', 'supercategory': 'M2A1Slammer'}]

In [7]:
import shutil
source_img = "/disk3/eric/dataset/Weapon_System/splits"

def copy_images(ann_dir,source_img_dir, target_img_dir):
    with open(ann_dir, 'r') as f:
        data = json.load(f)
    #--------------------------------
    images = data['images']
    
    #--- train 
    for i in images:
        #print(i['file_name'])
        z = i['file_name'].split(".")[0].replace("_png",".png")
        src_path = os.path.join(source_img_dir,z)
        dst_path = os.path.join(target_img_dir,z)
        shutil.copy(src_path,dst_path)
        
#--
copy_images(train_ann_dir, source_img, train_img_dir)
copy_images(valid_ann_dir, source_img, valid_img_dir)

In [8]:
def create_mask(image_info, annotations, output_folder):
    
    # Create an empty mask as a numpy array
    mask_np = np.zeros((image_info['height'], image_info['width']), dtype=np.uint16)
    
    for ann in annotations:
        if ann['image_id'] == image_info['id']:
                #print("#----- : ",ann['image_id'])
                # Extract segmentation polygon
                #ann['category_id'] +=1
                category_id = ann['category_id'] 
                
                for idx,seg in enumerate(ann['segmentation']):
                    # Convert polygons to a binary mask and add it to the main mask
                    rr, cc = skimage.draw.polygon(seg[1::2], seg[0::2], mask_np.shape)
                    mask_np[rr, cc] = category_id
                    
                    #object_number += 1 #We are assigning each object a unique integer value (labeled mask)

    # Save the numpy array as a TIFF using tifffile library
    #print(image_info['file_name'].split(".")[0].replace("_png",".png"))
    mask_path = os.path.join(output_folder, image_info['file_name'].split(".")[0].replace("_png",".png"))
    tifffile.imsave(mask_path, mask_np)

    image_info['file_name'] = image_info['file_name'].split(".")[0].replace("_png",".png") 

    #----

    print(f"Saved mask for {image_info['file_name']} to {mask_path}")

    return annotations

#--- train
with open(train_ann_dir, 'r') as f:
    data = json.load(f)
images = data['images']
annotations = data['annotations']

for img in images:
    # Create the masks
    create_mask(img, annotations, train_mask_dir)
#--- valid
with open(valid_ann_dir, 'r') as f:
    data = json.load(f)
images = data['images']
annotations = data['annotations']

for img in images:
    # Create the masks
    create_mask(img, annotations, valid_mask_dir)

Saved mask for Zamak__part_21.png to /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/Zamak__part_21.png
Saved mask for M2A1Slammer2__part_24.png to /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/M2A1Slammer2__part_24.png
Saved mask for Zamak__part_18.png to /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/Zamak__part_18.png
Saved mask for M5Sandstorm__part_17.png to /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/M5Sandstorm__part_17.png
Saved mask for T140Angara__part_8.png to /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/T140Angara__part_8.png
Saved mask for city_part_5.png to /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/city_part_5.png
Saved mask for M2A1Slammer2__part_23.png to /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/M2A1Slammer2__part_23.png
Saved mask for Zamak__part_23.png to /disk3/eric/dataset/Weapon_System/segmentation_pipes/train/labels/Z

/tmp/ipykernel_1191442/3938214246.py:23: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(mask_path, mask_np)
